# Let's do some inverse kinematics
Inverse kinematics is probably the core process in biomechanics.
It consists in getting body angles from a set of collected data. 

In this file, we investigate how perform such an analysis using biorbd. 
Since we can't use a real data collection system, the data are create though.

In [ ]:
# So first, let's import all the required modules 
from biorbd import Biorbd  # This is the core that will do all the calculation

from copy import copy  # Allows to copy arrays
import numpy  # Numpy is a python module that helps dealing with the mathematics of matrices and vectors
numpy.set_printoptions(precision=4, suppress=True)  # Make the printing of numpy matrices prettier

from matplotlib import pyplot

## First load a previously created bioMod file
This first section prepares Python and load a `.bioMod` file as shown in the `0-kinematic_model_creation` script.

In [ ]:
# Import and interpret the bioMod of the full body (created by the script '0-kinematic_model_creation' or `2.5-simple_upper_limb)
use_upper_limb = False

if use_upper_limb:
    model = Biorbd("models/SimpleUpperLimbWithInertia.bioMod")
    resting_pose = (0.15,)  # Define a position of the model that prints well
else:
    model = Biorbd("models/SimpleBody.bioMod")
    resting_pose = (0, 0, -0.15, 0.15, 0.7, -1, -1.11)  # Define a position of the model that prints well
    

# Define some useful information
n_q = model.nb_q  # The number of degrees of freedom in the model

## Creating some data
The next section generates data. 
This is to simulate collecting data on a real human.
The movement we simulate here is simply raising the arm vertical point down to horizontal pointing forward.
We use this opportunity to showcase how to perform forward kinematics, that is, computing the position of the end effector given the joint angles.

The strategy is to create a collection of position (stored in a matrix of "n degrees-of-freedom" by "n frames").
Then to replace some of the data points (namely those related to the arm) so it performs the required movement.
Finally, we will find the position of the skin markers for each of the data points (This is as if you were moving the avatar at a position, and the write down the 3d position of each blue dots). 

Remember that the arm in the SimpleBody.bioMod is the 4th degree-of-freedom so it is indexed "3" as the first index is "0".
The downward pointing is when $q[3] = 0.15$ radian and fronward point at $1.7$ radian. 
We simulate 100 frames in all


In [ ]:
# The movement we are going to simulate, for simplicity, is an arm raising
# First find the index of the arm x rotation in the vector of degree of freedom (this is determined by the model)
arm_index = model.dof_names.index("UPPER_ARM_RotX")
initial_pose = list(resting_pose)
final_pose = list(resting_pose)
final_pose[arm_index] = 1.7

# Let's declare some variables to make our life easy
number_frames = 100
frame_by_second = 100
with_noise = True

# Get us a time vector which will be useful while graphing
time_vector = numpy.linspace(0, number_frames / frame_by_second, number_frames)

# First create a static body at each time frame
# So let's have the full body at a starting posture (arm down and knee flexed)
# Note: the transpose ensures that it is a column vector (time will be on each column)
full_body_position = numpy.array([initial_pose]).transpose()

# Repeat that full body position, one for each frame
full_body_all_position = numpy.repeat(full_body_position, number_frames, axis=1)  # axis=1 is to repeat on columns

In [ ]:
# Now we can create an arm movement over time
arm_position_over_time = numpy.linspace(initial_pose[arm_index], final_pose[arm_index], number_frames)

# Add some noise as no data are perfect
if with_noise:
    noise = (numpy.random.rand(arm_position_over_time.shape[0]) - 0.5) / 20  # Noise of ±0.025
    arm_position_over_time_noised = arm_position_over_time + noise
else:
    arm_position_over_time_noised = arm_position_over_time

# And replace the arm position in the body positions 
full_body_all_position[arm_index, :] = arm_position_over_time_noised

In [ ]:
# If you want to see the movement you just simulated, you can run this cell
# Note: Due to Jupyter limitations, this may or may not cause issue
# Please also note that this cell will crash if runned from the binder

import bioviz

# Send the previously loaded model to the vizualizer (bioviz is not yet compatible with the latest Biorbd API, so we access the internal biorbd model)
viz = bioviz.Viz(loaded_model=model.internal)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.load_movement(full_body_all_position)

# Halt the program so you can admire the movement with the vizualiser. Closing the window should allow to continue
viz.exec()

In [ ]:
# Finally we can "collect" the position of each skin markers at each frame (the so-called forward kinematics). 
# This effectively simulates a data collection. Data could originate from 
# sofisticated equipment such as a Vicon system or it can be produced using
# more lay software such as Kinovea. The important bit is that the data are
# structured in a way that allows to subtract simulated data to real data
all_skin_markers = []
for f in range(number_frames):
    all_skin_markers.append(numpy.concatenate([mark.world[:, None] for mark in model.markers(full_body_all_position[:, f])], axis=1))


## Inverse kinematics
Inverse kinematics tries to find what are the positions that could have produced a set of given markers
In our case, these markers are the simulated one. 
In the real life, it would be collected data

The stategy is to simulate a lot a different position and find the one that reduces the most the error between the measured markers with the simulated one. Then to repeat for each frame.

In [ ]:
# First let's define a function that takes in a position (q) and spits out a matrix of markers (3 x nMarkers)
def forward_kinematics(q):
    return numpy.concatenate([mark.world[:, None] for mark in model.markers(q)], axis=1)
    
# Define also a jacobian function
def marker_jacobian(q):
    return numpy.concatenate([mark.jacobian() for mark in model.markers(q)])

# Finally define the inverse kinematics function base on 
# https://en.wikipedia.org/wiki/Inverse_kinematics#The_Jacobian_inverse_technique
def inverse_kinematics(real_markers, q_init, number_of_iterations=5):
    for i in range(number_of_iterations):
        # Get the marker and jacobian at q_init
        simulated_markers = forward_kinematics(q_init[:, 0])
        jacobian_matrix = marker_jacobian(q_init[:, 0])

        # Put markers into a column vector (3*nMarkers x 1)
        simulated_markers = simulated_markers.T.flatten()[:, numpy.newaxis]
        real_markers = real_markers.T.flatten()[:, numpy.newaxis]

        # Compute the step length to perform. Reminder: @ is a matrix multiplication operator
        q_advance = numpy.linalg.pinv(jacobian_matrix) @ (simulated_markers - real_markers)

        # Advance the q_init and repeat until number_of_iterations is exhausted
        q_init -= q_advance
    return q_init

In [ ]:
# The let's test these functions and print the result to make sure it works as expected
q = numpy.array(final_pose)
print(f"The markers when the body is at q={q} are:")
print("")
print(forward_kinematics(q))
print("")
print("The marker jacobian matrix at the same position is:")
print(marker_jacobian(q))

In [ ]:
# Perform the inverse kinematics for the frame 0
frame = 0
number_of_iterations = 5  # Number of Newton iterations

# First get the "collected" real markers data
# real_markers = all_skin_markers[frame].transpose().flatten()[:, numpy.newaxis]
real_markers = all_skin_markers[frame]

# Perform the inverse kinematics
q_init = numpy.zeros((n_q, 1))  # Initialize to a poor position (all zeros)
q_inverse = inverse_kinematics(real_markers, q_init, number_of_iterations)
    
# Compare the results with the expected value (that we know because we created the data ourselves)
q_real = full_body_all_position[:, frame][:, numpy.newaxis]  # Get the reference answer

print(f"The expected answer is: {q_real.transpose()}")
print(f"The computed answer is: {q_inverse.transpose()}")
print(f"The RMSE between them is: {numpy.sqrt(sum((q_inverse - q_real)**2))}")

In [ ]:
# Now that we know how to do it for one frame, we can do it at each frame
# But that time, we can use a better q_init guess for the next frames as we know the 
# body won't move much between frames

number_of_iterations = 5  # Number of Newton iterations

q_init = numpy.zeros((n_q, 1))  # Initialize to a poor position (all zeros)
q_inverse = numpy.ndarray((n_q, number_frames))
for frame in range(len(all_skin_markers)):
    # First get the "collected" real markers data
    real_markers = all_skin_markers[frame]

    # Perform the inverse kinematics
    q_inverse[:, frame] = inverse_kinematics(real_markers, q_init, number_of_iterations)[:, 0]
    
    # Take the previous position as the starting point. 
    q_init = copy(q_inverse[:, frame][:, numpy.newaxis])
    number_of_iterations = 2  # We don't need to perform as much iteration now that we have a good starting point

# Now print the sum RMSE
q_real = full_body_all_position  # Get the reference answer
print(f"The sum RMSE is: {sum(numpy.sqrt(sum((q_inverse - q_real)**2)))}")

In [ ]:
# We can finally graph the real kinematics of the arm against the reconstructed
pyplot.title("Arm angle against time")
pyplot.ylabel("Arm angle (rad)")
pyplot.xlabel("Time (s)")

pyplot.plot(time_vector, q_real[arm_index, :])
pyplot.plot(time_vector, q_inverse[arm_index, :])

In [ ]:
# Let's finally save the results so it can be used in the inverse dynamics script
import os
if not os.path.isdir("results"):
    os.mkdir("results")

numpy.save("results/time_vector.npy", time_vector)
if use_upper_limb:
    numpy.save("results/inverse_kinematics_upper_limb.npy", q_inverse)
else:
    numpy.save("results/inverse_kinematics.npy", q_inverse)